In [1]:
import pandas as pd

df = pd.read_csv("./data/raw/train_cax.csv")
oof = pd.read_csv("./data/result/oof.csv")

dt = pd.read_csv("./data/raw/test_cax.csv")
ds = pd.read_csv("./data/result/LSTM_submit.csv")

dt["label"] = list(ds["label"])
df["pred"] = oof["pred_label"]

In [2]:
from math import floor

df["leak"] = df["GR"]*100
df["leak"] = df["leak"] - df["leak"].apply(floor)
df["leak"] = df["leak"].apply(lambda x: round(x, 11))

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["label"]==i]["leak"].value_counts().keys())
        b = set(df[df["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))

0 1 	 1860 174921 	 0
0 2 	 1860 16100 	 0
0 3 	 1860 155569 	 0
0 4 	 1860 154186 	 0
1 2 	 174921 16100 	 0
1 3 	 174921 155569 	 0
1 4 	 174921 154186 	 0
2 3 	 16100 155569 	 0
2 4 	 16100 154186 	 0
3 4 	 155569 154186 	 0


In [3]:
s = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(df[df["pred"]==i]["leak"].value_counts().keys())
        b = set(df[df["pred"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        s |= a&b
        
len(s)

0 1 	 2015 175370 	 140
0 2 	 2015 16806 	 7
0 3 	 2015 156135 	 26
0 4 	 2015 154786 	 38
1 2 	 175370 16806 	 228
1 3 	 175370 156135 	 555
1 4 	 175370 154786 	 492
2 3 	 16806 156135 	 337
2 4 	 16806 154786 	 373
3 4 	 156135 154786 	 398


2362

In [4]:
1-len(df[df["label"]!=df["pred"]])/len(df)

0.9958161363636364

In [5]:
from tqdm import tqdm_notebook as tqdm
d = dict()
for c in tqdm(list(s)):
    k = df[df["leak"]==c]["pred"].value_counts().keys()[0]
    d[c] = k

In [6]:
res = []
for k, v in tqdm(zip(df["leak"], df["pred"]), total=len(df)):
    if k in d:
        res.append(d[k])
    else:
        res.append(v)

In [7]:
df["res"] = res
print(1-len(df[df["label"]!=df["res"]])/len(df))

0.9964427272727273


In [8]:
dt["leak"] = dt["GR"]*100
dt["leak"] = dt["leak"] - dt["leak"].apply(floor)
dt["leak"] = dt["leak"].apply(lambda x: round(x, 11))

ss = set()

for i in range(5):
    for j in range(i+1,5):
        a = set(dt[dt["label"]==i]["leak"].value_counts().keys())
        b = set(dt[dt["label"]==j]["leak"].value_counts().keys())
        print(i,j,"\t",len(a),len(b),"\t",len(a&b))
        ss |= a&b
        
len(ss)

0 1 	 1947 89411 	 89
0 2 	 1947 8463 	 4
0 3 	 1947 77985 	 19
0 4 	 1947 80268 	 11
1 2 	 89411 8463 	 94
1 3 	 89411 77985 	 206
1 4 	 89411 80268 	 200
2 3 	 8463 77985 	 129
2 4 	 8463 80268 	 136
3 4 	 77985 80268 	 174


985

In [9]:
dd = dict()
for c in tqdm(list(ss)):
    k = dt[dt["leak"]==c]["label"].value_counts().keys()[0]
    dd[c] = k

In [10]:
res = []
for k, v in tqdm(zip(dt["leak"], dt["label"]), total=len(dt)):
    if k in dd:
        res.append(dd[k])
    else:
        res.append(v)
        
dt["label"] = res

In [11]:
dt[["unique_id","label"]].to_csv("./data/result/processed.csv", index=False)

In [12]:
from random import choice

idx = df[df["label"]!=df["res"]].index
k = choice(list(idx))

df.iloc[k-10:k+10,:]

,row_id,well_id,GR,label,pred,leak,res
1823553,853,1657,114.424302,3,3,0.430158,3
1823554,854,1657,103.884302,3,3,0.430158,3
1823555,855,1657,107.584302,3,3,0.430158,3
1823556,856,1657,112.954302,3,3,0.430158,3
1823557,857,1657,91.124302,3,3,0.430158,3
1823558,858,1657,101.564302,3,3,0.430158,3
1823559,859,1657,97.904302,3,3,0.430158,3
1823560,860,1657,117.364302,3,3,0.430158,3
1823561,861,1657,121.554302,3,3,0.430158,3
1823562,862,1657,106.568972,1,3,0.897176,1
